In [1]:
import time
import os
import pandas as pd
from gensim.models import Word2Vec

In [2]:
import multiprocessing
WORKERS = multiprocessing.cpu_count()
print("Number of workers:",WORKERS)

Number of workers: 16


In [3]:
input_dir = "../input/amazon_review_polarity_csv/"

In [4]:
train_data = pd.read_csv(input_dir+"train_data_processed.csv")
test_data = pd.read_csv(input_dir+"test_data_processed.csv")
total_examples = train_data.shape[0]*2+test_data.shape[0]*2
del(train_data, test_data)

In [5]:
class MySentences(object):
    
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            if fname.endswith(".csv"):
                for line in open(os.path.join(self.dirname, fname)):
                    yield line.split()

In [6]:
sentences = MySentences('../input/amazon_review_polarity_csv_chunks/')

In [7]:
def word2vec_generator(sentences, size, window, total_examples, workers, destination_folder = "./word2vec_models/"):
    '''
    A function to generate word2vec model
    '''    
    # create a word2vec model
    model = Word2Vec(sentences, size=size, window=window, sg= 1, min_count=1, workers=workers)
    # train model
    model.train(sentences, total_examples=total_examples, epochs=5)
    # file path
    fname = destination_folder + "word2vec_size_" + str(size) + "_window_" + str(window) + ".mdl"
    # save the model
    model. wv.save_word2vec_format(fname)

In [8]:
list_of_sizes = [100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 300, 300, 300, 300, 300]
list_of_window_sizes = [5,10,15,20,25,5,10,15,20,25,5,10,15,20,25]
model_info = pd.DataFrame({"index":list(range(len(list_of_sizes))),
                           "size": list_of_sizes,
                           "window": list_of_window_sizes})
model_info

,index,size,window
0,0,100,5
1,1,100,10
2,2,100,15
3,3,100,20
4,4,100,25
5,5,200,5
6,6,200,10
7,7,200,15
8,8,200,20
9,9,200,25


In [9]:
selected_indexes = [2,3,4,5,6,7,8,9,10,11,12,13,14]

In [10]:
for index in selected_indexes:
    t_start = time.time()
    size = model_info[model_info.index == index]["size"].values[0]
    window = model_info[model_info.index == index]["window"].values[0]
    word2vec_generator(sentences, size, window, total_examples, WORKERS)
    t_end = time.time()
    print("Time taken to save model with vector size {} and window size {}: {:.2f} mins"\
          .format(str(size),str(window),(t_end-t_start)/60))

Time taken to save model with vector size 100 and window size 15: 78.58 mins
Time taken to save model with vector size 100 and window size 20: 95.02 mins
Time taken to save model with vector size 100 and window size 25: 110.69 mins
Time taken to save model with vector size 200 and window size 5: 54.03 mins
Time taken to save model with vector size 200 and window size 10: 74.32 mins
Time taken to save model with vector size 200 and window size 15: 96.97 mins
Time taken to save model with vector size 200 and window size 20: 118.37 mins
Time taken to save model with vector size 200 and window size 25: 137.14 mins
Time taken to save model with vector size 300 and window size 5: 60.15 mins
Time taken to save model with vector size 300 and window size 10: 87.16 mins
Time taken to save model with vector size 300 and window size 15: 115.71 mins
Time taken to save model with vector size 300 and window size 20: 140.28 mins
Time taken to save model with vector size 300 and window size 25: 169.95 